In [15]:
import pandas as pd
import spacy
from spacy.training import Example

In [16]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import pandas as pd
import random
def extract_skills(skills_str):
    return [skill.strip() for skill in skills_str.split(',')]
# Load the spaCy English tokenizer
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger'])

# Add the NER pipe if it doesn't exist
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

# Add the SKILL label to the NER model
ner.add_label('SKILL')

# Read the CSV file
df = pd.read_csv('./job_skills.csv')

# Generate synthetic training data
TRAIN_DATA = []
for _, row in df.iterrows():
    # convert to string 
    row['job_skills'] = str(row['job_skills'])
    skills = extract_skills(row['job_skills'])
    for skill in skills:
        # Create a synthetic sentence with the skill in it
        text = f"I have experience in {skill}."
        # Find the start and end indices of the skill in the context
        start = text.find(skill)
        end = start + len(skill)
        # Append the text and the entity to the training data
        TRAIN_DATA.append((text, {'entities': [(start, end, 'SKILL')]}))

# Shuffle the training data
random.shuffle(TRAIN_DATA)

# Train the NER model
optimizer = nlp.resume_training()
for i in range(10):  # Number of training iterations
    random.shuffle(TRAIN_DATA)
    losses = {}
    # Batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for j in range(len(texts)):
            doc = nlp.make_doc(texts[j])
            example.append(Example.from_dict(doc, annotations[j]))
        nlp.update(example, drop=0.5, losses=losses)
    print(f"Losses at iteration {i}", losses)

# Save the model to disk
nlp.to_disk('./model')


c:\Users\Marouane\Documents\NLP_ PROJECT V1.01\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I have experience in QoS." with entities "[(21, 24, 'SKILL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Marouane\Documents\NLP_ PROJECT V1.01\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I have experience in Salesforce Developer I." with entities "[(21, 43, 'SKILL')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Marouane\Documents\NLP_ PROJECT V1.01\env\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the tex